In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import akita_utils_forplotting as utils
import numpy as np
import pandas as pd
import sys, os, psutil, time
import matplotlib.pyplot as plt
import pysam
    
from skimage.transform import resize


#from optparse import OptionParser


nt = ['A', 'T', 'C', 'G']

2023-11-10 11:25:49.957891: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-11-10 11:25:49.957927: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-11-10 11:25:52.090658: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-11-10 11:25:52.090691: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-11-10 11:25:52.090722: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ruby.gladstone.internal): /proc/driver/nvidia/version does not exist
2023-11-10 11:25:52.090909: I tensorflow/core/platfor

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 sequence (InputLayer)          [(None, 1048576, 4)  0           []                               
                                ]                                                                 
                                                                                                  
 stochastic_reverse_complement   ((None, 1048576, 4)  0          ['sequence[0][0]']               
 (StochasticReverseComplement)  , ())                                                             
                                                                                                  
 stochastic_shift (StochasticSh  (None, 1048576, 4)  0           ['stochastic_reverse_complement[0
 ift)                                                            ][0]']                     

                                                                                                  
 re_lu_8 (ReLU)                 (None, 4096, 96)     0           ['max_pooling1d_7[0][0]']        
                                                                                                  
 conv1d_8 (Conv1D)              (None, 4096, 96)     46080       ['re_lu_8[0][0]']                
                                                                                                  
 batch_normalization_8 (BatchNo  (None, 4096, 96)    384         ['conv1d_8[0][0]']               
 rmalization)                                                                                     
                                                                                                  
 max_pooling1d_8 (MaxPooling1D)  (None, 2048, 96)    0           ['batch_normalization_8[0][0]']  
                                                                                                  
 re_lu_9 (

 add_2 (Add)                    (None, 512, 96)      0           ['add_1[0][0]',                  
                                                                  'dropout_2[0][0]']              
                                                                                                  
 re_lu_17 (ReLU)                (None, 512, 96)      0           ['add_2[0][0]']                  
                                                                                                  
 conv1d_17 (Conv1D)             (None, 512, 48)      13824       ['re_lu_17[0][0]']               
                                                                                                  
 batch_normalization_17 (BatchN  (None, 512, 48)     192         ['conv1d_17[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 re_lu_18 

 batch_normalization_25 (BatchN  (None, 512, 48)     192         ['conv1d_25[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 re_lu_26 (ReLU)                (None, 512, 48)      0           ['batch_normalization_25[0][0]'] 
                                                                                                  
 conv1d_26 (Conv1D)             (None, 512, 96)      4608        ['re_lu_26[0][0]']               
                                                                                                  
 batch_normalization_26 (BatchN  (None, 512, 96)     384         ['conv1d_26[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 dropout_7

                                                                                                  
 batch_normalization_32 (BatchN  (None, 512, 512, 48  192        ['conv2d_4[0][0]']               
 ormalization)                  )                                                                 
                                                                                                  
 dropout_9 (Dropout)            (None, 512, 512, 48  0           ['batch_normalization_32[0][0]'] 
                                )                                                                 
                                                                                                  
 add_9 (Add)                    (None, 512, 512, 48  0           ['symmetrize2d_1[0][0]',         
                                )                                 'dropout_9[0][0]']              
                                                                                                  
 symmetriz

 ormalization)                  )                                                                 
                                                                                                  
 dropout_12 (Dropout)           (None, 512, 512, 48  0           ['batch_normalization_38[0][0]'] 
                                )                                                                 
                                                                                                  
 add_12 (Add)                   (None, 512, 512, 48  0           ['symmetrize2d_4[0][0]',         
                                )                                 'dropout_12[0][0]']             
                                                                                                  
 symmetrize2d_5 (Symmetrize2D)  (None, 512, 512, 48  0           ['add_12[0][0]']                 
                                )                                                                 
          

In [3]:
import h5py
import pyBigWig
import intervaltree
import basenji
import matplotlib.pyplot as plt

# hic imports
import cooler
from cooltools.lib.numutils import observed_over_expected, adaptive_coarsegrain
from cooltools.lib.numutils import interpolate_bad_singletons, set_diag, interp_nan
from astropy.convolution import Gaussian2DKernel
from astropy.convolution import convolve

import collections 

In [4]:
import cooltools.lib.plotting
from matplotlib.colors import LogNorm
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.ticker import EngFormatter
import seaborn as sns


In [5]:
from scipy.ndimage import gaussian_filter, gaussian_filter1d, uniform_filter
from astropy.convolution import Gaussian2DKernel

In [6]:
#REMEMBER!!!!!!! +1MB SEQUENCE GOES IN, BUT MAKES PREDICTION FOR 917504BP
#1MB CORRESPONDS TO 512, 2048 BP BINS THAT GETS CUT DOWN TO 448 BINS 

In [7]:
#TOTAL AKITA INPUT LENGTH 

akita_seq_len=1048576


#HG38 REF FA
hg38_file = '/pollard/home/shzhang/akita/run_akita/refs/hg38.fa'
fasta_hg38 = pysam.Fastafile(hg38_file)

#GENCODE
gene_annot = pd.read_csv('/pollard/data/genetics/GENCODE/release32/gencode.v32.genes_only.bed', sep = '\t', names = ['CHROM', 'start_gene', 'end_gene', 'Gene'])
gene_annot[gene_annot.CHROM != 'chrM']

,CHROM,start_gene,end_gene,Gene
0,chr1,11869,14409,DDX11L1
1,chr1,14404,29570,WASH7P
2,chr1,17369,17436,MIR6859-1
3,chr1,29554,31109,MIR1302-2HG
4,chr1,30366,30503,MIR1302-2
...,...,...,...,...
60567,chrY,57184101,57197337,IL9R
60568,chrY,57190738,57208756,AJ271736.1
60569,chrY,57201143,57203357,WASIR1
60570,chrY,57207346,57212230,WASH6P


In [8]:
#to do normalization
#functions from ohther files 
ModelSeq = collections.namedtuple('ModelSeq', ['chr', 'start', 'end', 'label'])

def read_blacklist(blacklist_bed, black_buffer=20):
    """Construct interval trees of blacklist
     regions for each chromosome."""
    black_chr_trees = {}

    if blacklist_bed is not None and os.path.isfile(blacklist_bed):
        for line in open(blacklist_bed):
            a = line.split()
            chrm = a[0]
            start = max(0, int(a[1]) - black_buffer)
            end = int(a[2]) + black_buffer

        if chrm not in black_chr_trees:
            black_chr_trees[chrm] = intervaltree.IntervalTree()

        black_chr_trees[chrm][start:end] = True

    return black_chr_trees


In [9]:
utils.half_patch_size

524288

In [10]:

def format_ticks(ax, x=True, y=True, rotate=True):
    if y:
        ax.yaxis.set_major_formatter(bp_formatter)
    if x:
        ax.xaxis.set_major_formatter(bp_formatter)
        ax.xaxis.tick_bottom()
    if rotate:
        ax.tick_params(axis='x',rotation=45)

In [11]:
def read_fasta(fa):
    #read in fasta and return sequence only 
    seq=''
    with open(fa, "r") as fa_file:
        for line in fa_file:
            if not (line.startswith('>')):
                seq += line.strip()
    fa_file.close()                
    return(seq)
    

In [12]:
#data directories
date = '20231110_v1'
d='/pollard/home/shzhang/akita_variant_scoring/akdemir_collab/'
data_dir = d + 'data/'
fig_dir = f'{d}figs/{date}/'


#two folders are colab_fasta and new_fasta
list_mcool=os.listdir(data_dir + 'mcool/')
list_mcool

['SMF-12-23_walk_9_MDM2_containing.fa.mcool',
 'SMF-12-23_walk_8_MDM2_containing.fa.mcool',
 'SMF-12-12_walk_3_KRAS_containing.fa.mcool',
 'SMF-12-23_walk_9_GLI1_containing.fa.mcool',
 'SMF-12-12_walk_4_KRAS_containing.fa.mcool',
 'SMF-12-12_walk_5_KRAS_containing.fa.mcool',
 'SMF-12-17_walk_7_MDM2_containing.fa.mcool',
 'SMF-12-23_walk_11_GLI1_containing.fa.mcool',
 'SMF-12-12_walk_4_GLI1_containing.fa.mcool',
 'SMF-12-17_walk_4_GLI1_containing.fa.mcool',
 'SMF-12-23_walk_11_MDM2_containing.fa.mcool',
 'SMF-12-17_walk_9_MDM2_containing.fa.mcool',
 'SMF-12-23_walk_10_MDM2_containing.fa.mcool',
 'SMF-12-17_walk_8_MDM2_containing.fa.mcool',
 'SMF-12-17_walk_5_GLI1_containing.fa.mcool',
 'SMF-12-17_walk_3_GLI1_containing.fa.mcool']

In [13]:
#make akita predictions for each walk and plot 
def akita_pred(mcool):
    i=mcool.replace('.fa.mcool', '.fa')
    print(i)
    bed_name=i[:-3] +'.bed'
    name_parts = i.rsplit('_', 2)

    gencode_coord=gene_annot[gene_annot.Gene == name_parts[1]]    
    gene_len=abs(gencode_coord.end_gene-gencode_coord.start_gene)
    gene_mid_coord=np.floor(gencode_coord.start_gene + (gene_len/2))
    #print('gene mid coord', gene_mid_coord)
    #print('gencode coord', gencode_coord)
    
    #read in bed file and figure out where to center the file 
    bed=pd.read_csv(data_dir+ 'new_fasta/walks.bed/' + bed_name, sep='\t', header=None)
    #rename columns
    bed.columns = ['chr', 'start', 'end', 'strand', '4', '5']
    bed['chr'] = bed['chr'].astype('str') 
    #get spans of each walk 
    bed['span']=abs(bed['end']-bed['start']) 
    
    #print('bed span', np.sum(bed['span']))


    #get the row where gene_mid_coord 
    #print(gencode_coord['CHROM'])
    condition = ((bed['chr']==gencode_coord.CHROM.values[0])
                & (bed['start'] <= int(gene_mid_coord)) & (int(gene_mid_coord) <= bed['end']) ) 
    #in cases where the middle of the gene shows up more than once, just takes first instance 
    row_midgene = (condition.index[condition].tolist())[0]
    #print('row_midgene index', row_midgene)
                

    #get span for the entire walk up to the middle of the gene 
    bed_row=bed.iloc[row_midgene, :]
    #print('bed_Row', bed_row)
    #print('gene row span', np.sum(bed_row['span']))
    #print('gene_mid_Coord', gene_mid_coord)
    if row_midgene>0:
        bed_begin=bed.iloc[0:row_midgene, :]
        #print('bed_begin', bed_begin)
        first_half_span=(np.sum(bed_begin['span'])) + int(gene_mid_coord-bed_row['start'])
        bed_before=pd.concat([bed_begin, bed_row])

    else:
        first_half_span=int(gene_mid_coord-bed_row['start'])
        bed_before=bed_row
    #print('bed_before', bed_row)
            
    #then, get span for entire walk from middle of gene to end 
    if row_midgene<(len(bed)-1):
        bed_end=bed.iloc[(row_midgene+1):len(bed), :]
        #print('bed_end', bed_end)
        second_half_span=(np.sum(bed_end['span'])) + int(bed_row['end']-gene_mid_coord)
    else:
        second_half_span=int(bed_row['end']-gene_mid_coord)

        
    if 1==1:   
        #read in fasta file 
        seq=read_fasta(data_dir + 'new_fasta/walks.fasta/' + i)

     
    
        #define ALT from fasta sequences given to us       
        #check length of walk start to middle of gene, must be at lesat half of akita_seq_len
        mid_seq_idx=0
        gene_bin_start=0
        gene_bin_end=0
        seq_start=0
        seq_end=0
        gene_half=round(gene_len/2)
        
        #at least right now, no sequences need to be padded on both left and right so this works.
        #in future, if that's the case, we will need to change this
        if first_half_span<=(utils.half_patch_size):
            print('starting from left most sequence')
            
            seq_start=0
            seq_end=utils.MB
            ALT_seq=seq[0:utils.MB]
            
            gene_bin_start=utils.get_bin(first_half_span-gene_half)
            gene_bin_end=utils.get_bin(first_half_span+gene_half)
          
        
            #get REF POS 
            #POS = int(gene_mid_coord-utils.half_patch_size) + (utils.half_patch_size-first_half_span)
            POS = int(gene_mid_coord-first_half_span)

        
           
        elif second_half_span<=(utils.half_patch_size):          
            print('ending at rightmost sequence ')
            
            seq_start=(len(seq)-utils.MB)
            seq_end=len(seq)       
            ALT_seq=seq[seq_start:seq_end]
            
            gene_bin_start=utils.get_bin((utils.MB-second_half_span-gene_half))
            gene_bin_end=utils.get_bin((utils.MB-second_half_span+gene_half))

                    
            #get REF POS 
            POS = int(gene_mid_coord-utils.half_patch_size) - (utils.half_patch_size-second_half_span)

            
        else:  
            seq_start=int(first_half_span-(utils.half_patch_size))
            seq_end=int(seq_start+utils.MB)  
            print(seq_start)
            print(seq_end)
            print(len(seq))
            ALT_seq=seq[seq_start:seq_end]

            gene_bin_start=utils.get_bin(utils.half_patch_size - round(gene_len/2))
            gene_bin_end=utils.get_bin(utils.half_patch_size + round(gene_len/2))
            
                    
            #get REF POS 
            POS = int(gene_mid_coord-utils.half_patch_size)
            
            
            
        #get REF                   
        #define start and end coordinates of reference matrices
        #currently, just centered around the middle of the gene of interest        
        CHR = gencode_coord.CHROM.values[0]
        END= int(POS+utils.seq_length)
        REF_seq = fasta_hg38.fetch(CHR, POS, END).upper()
            
        ####################
        
        assert len(REF_seq) == utils.seq_length, 'REF_seq is not right length'
        assert len(ALT_seq) == utils.seq_length, 'ALT_seq is not right length'
        
#         print('gene bin start', gene_bin_start)
#         print('gene bin end', gene_bin_end)
        
#         print('seq start', seq_start)
#         print('seq end', seq_end)
#         print('seq len', len(seq))

    
        #this is where akita is actually run, output vector size is (99681,)
        REF_vector, ALT_vector = utils.vector_from_seq(REF_seq), utils.vector_from_seq(ALT_seq)
        REF_mat, ALT_mat = utils.mat_from_vector(REF_vector), utils.mat_from_vector(ALT_vector)


        ALT_label=i[:-3]
        REF_label='{}:{}-{}'.format(CHR, POS, END)
        return(REF_mat, ALT_mat, REF_label, ALT_label, 
               gene_bin_start, gene_bin_end, gene_len, 
               CHR, seq_start, seq_end)


#         plot_complex_pred(ALT_mat, REF_mat, ALT_label, REF_label, gene_len)
            
        
    else:
        print('error!!!! in file ', i )

In [14]:
#below is to normalize original hi-c maps

In [15]:
def normalize_hic(file_path, seqs_info, resolution, clip, crop_bp, kernel_stddev, pool_width, 
                  no_log, crop, as_obsexp, global_obsexp, diagonal_offset, norm_type, output_file):
    genome_hic_file = f'{file_path}::resolutions/{resolution}'
    print(genome_hic_file)
    #bed file of sequences that we want from the initial hic file 


    #path to where output file should be stored
    seqs_hic_file = output_file
    
    ####################
    # read model sequences
    model_seqs = []
    model_seqs.append(ModelSeq(seqs_info[0],int(seqs_info[1]),int(seqs_info[2]),None))

    # compute dimensions
    num_seqs = len(model_seqs)
    seq_len_nt = model_seqs[0].end - model_seqs[0].start
    seq_len_pool = seq_len_nt // pool_width

    if crop_bp == 0:
        seq_len_crop = seq_len_pool
    else:
        crop_start = crop_bp // pool_width
        crop_end = seq_len_pool - crop_start
        seq_len_crop = seq_len_pool - 2*crop_start

    # compute upper triangular indexes
    triu_tup = np.triu_indices(seq_len_crop, diagonal_offset)
    seq_len_nodiag = seq_len_crop - diagonal_offset
    seq_len_hic = seq_len_nodiag*(seq_len_nodiag + 1) // 2


    if kernel_stddev > 0:
        # initialize Gaussian kernel
        kernel = Gaussian2DKernel(x_stddev=kernel_stddev)
        #kernel = Gaussian2DKernel(x_stddev=kernel_stddev, x_size=kernel_size)
    else:
        kernel = None
        
        
    # open genome coverage file
    genome_hic_cool = cooler.Cooler(genome_hic_file)
    #check for "chr" prefix
    chr_pre = 'chr1' in genome_hic_cool.chromnames
    # assert that resolution matches
    assert(pool_width == genome_hic_cool.info['bin-size'])
    
    
        # initialize sequences coverage file
    #seqs_hic_open = h5py.File(seqs_hic_file, 'w')
    #seqs_hic_open.create_dataset('targets', shape=(num_seqs, seq_len_hic), dtype='float16')
    targets=np.zeros((num_seqs, seq_len_hic))

    # for each model sequence
    for si in range(num_seqs):
        mseq = model_seqs[si]
        print(mseq)
        if 1==1:
          # pull hic values
            if chr_pre:
                mseq_str = '%s:%d-%d' % (mseq.chr[3:], mseq.start, mseq.end)
            else:
                mseq_str = '%s:%d-%d' % (mseq.chr, mseq.start, mseq.end)
                #print('mseq_str:', mseq_str)

            seq_hic_raw = genome_hic_cool.matrix(balance=norm_type).fetch(mseq_str)
            seq_hic_nan = np.isnan(seq_hic_raw)
            num_filtered_bins = np.sum(np.sum(seq_hic_nan,axis=0) == len(seq_hic_nan))


            if num_filtered_bins > (.5*len(seq_hic_nan)):
                print("WARNING: %s >50%% bins filtered, check:  %s. " % (genome_hic_file, mseq_str))


          # clip first diagonals and high values
            clipval = np.nanmedian(np.diag(seq_hic_raw,diagonal_offset))
            for i in range(-diagonal_offset+1,diagonal_offset):
                set_diag(seq_hic_raw, clipval, i)
            seq_hic_raw = np.clip(seq_hic_raw, 0, clipval)
            seq_hic_raw[seq_hic_nan] = np.nan

          # adaptively coarsegrain based on raw counts
            seq_hic_smoothed = adaptive_coarsegrain(
                seq_hic_raw,
                #use KR in this case 
                genome_hic_cool.matrix(balance=norm_type).fetch(mseq_str),
                cutoff=2, max_levels=8)
            seq_hic_nan = np.isnan(seq_hic_smoothed)
          #todo: pass an option to add a certain pseudocount value, or the minimum nonzero value

            #THIS REQUIRES SOME PREKNOWN EXPECTED HIC COUNTS FILE WHICH WE DON'T HAVE 
            if as_obsexp:
                # compute obs/exp        
                if global_obsexp: # compute global obs/exp
                    exp_chr = genome_hic_expected.iloc[ genome_hic_expected['chrom'].values ==mseq.chr][0:seq_len_pool]
                    if len(exp_chr)==0: 
                          raise ValueError('no expected values found for chr:'+mseq.chr)
                    exp_map= np.zeros((seq_len_pool,seq_len_pool))
                    for i in range(seq_len_pool):
                        set_diag(exp_map,exp_chr['balanced.avg'].values[i],i)
                        set_diag(exp_map,exp_chr['balanced.avg'].values[i],-i)
                    seq_hic_obsexp = seq_hic_smoothed / exp_map
                    for i in range(-diagonal_offset+1, diagonal_offset): set_diag(seq_hic_obsexp,1.0,i)
                    seq_hic_obsexp[seq_hic_nan] = np.nan          

                else: # compute local obs/exp
                    seq_hic_obsexp = observed_over_expected(seq_hic_smoothed, ~seq_hic_nan)[0]

                # TAKE NATURAL LOG 
                #AND CLIP TO (-2,2)
                #AND ALSO INTERPOLATE 
                if no_log==False:
                    seq_hic_obsexp = np.log(seq_hic_obsexp)
                    if clip is not None:
                        seq_hic_obsexp = np.clip(seq_hic_obsexp, -clip, clip)
                    seq_hic_obsexp = interp_nan(seq_hic_obsexp)
                    for i in range(-diagonal_offset+1, diagonal_offset): set_diag(seq_hic_obsexp, 0,i)

                else:
                    if clip is not None:
                        seq_hic_obsexp = np.clip(seq_hic_obsexp, 0, clip)
                    seq_hic_obsexp = interp_nan(seq_hic_obsexp)
                    for i in range(-diagonal_offset+1, diagonal_offset): set_diag(seq_hic_obsexp, 1,i)

                # apply kernel
                if kernel is not None:
                    seq_hic = convolve(seq_hic_obsexp, kernel)
                else:
                    seq_hic = seq_hic_obsexp

            else:
                # interpolate all missing bins
                seq_hic_interpolated = interp_nan(seq_hic_smoothed)

                # rescale, reclip
                seq_hic = 100000*seq_hic_interpolated
                clipval = np.nanmedian(np.diag(seq_hic,diagonal_offset))
                for i in range(-diagonal_offset+1, diagonal_offset):
                    set_diag(seq_hic,clipval,i)
                seq_hic = np.clip(seq_hic, 0, clipval)

                #extra smoothing. todo pass kernel specs
                if kernel is not None:
                    seq_hic = convolve(seq_hic, kernel)
        else:
        #except ValueError:
            print("WARNING: %s doesn't see %s. Setting to all zeros." % (genome_hic_file, mseq_str))
            seq_hic = np.zeros((seq_len_pool,seq_len_pool), dtype='float16')

        # crop
        if crop_bp > 0:
            seq_hic = seq_hic[crop_start:crop_end,:]
            seq_hic = seq_hic[:,crop_start:crop_end]

        # unroll upper triangular
        seq_hic = seq_hic[triu_tup]
        print(seq_hic.shape)

        # write
        #seqs_hic_open['targets'][si,:] = seq_hic.astype('float16')
        targets[si,:] = seq_hic.astype('float16')

    # close sequences coverage file
    #seqs_hic_open.close()
    

    targets=targets.reshape(targets.shape[1])       
    targets_mat = utils.from_upper_triu(targets,seq_len_pool,2)

    
    return(targets_mat)




In [16]:
# # Define the kernel function (e.g., Gaussian)
# def kernel(x, bandwidth):
#     return np.exp(-0.5 * (x / bandwidth)**2) / (bandwidth * np.sqrt(2 * np.pi))

In [17]:
def condense_matrices(target_size, original_size, akita_pred_matrix):
    
    original_matrix=akita_pred_matrix.copy()
    
    target_matrix = np.zeros((target_size, target_size))
    for target_row in range(target_size):
        for target_col in range(target_size):
            # Calculate the range of original bins to consider
            min_original_row = target_row * original_size // target_size
            max_original_row = (target_row + 1) * original_size // target_size
            min_original_col = target_col * original_size // target_size
            max_original_col = (target_col + 1) * original_size // target_size

            # Crop the last row and column by applying crop_weight
            cropped_matrix = original_matrix[:max_original_row, :max_original_col]
            cropped_matrix[-1, :] *= crop_weight
            cropped_matrix[:, -1] *= crop_weight
            
            kernel_std=2
            #kernel = Gaussian2DKernel(x_stddev=kernel_std)

            #print('cropped matrix shape', cropped_matrix.shape)
            # Apply 2D Gaussian smoothing
            smoothed_matrix = gaussian_filter(cropped_matrix, sigma=kernel_std)
            #smoothed_matrix=uniform_filter(original_matrix, size=3, mode='nearest', cval=np.nan)

            # Store the smoothed value in the target matrix
            target_matrix[target_row, target_col] = smoothed_matrix[-1, -1]
            #print(smoothed_matrix.shape)
            
            #renormalize so mean values still same across both matrices 
            target_matrix=(target_matrix/np.nanmax(target_matrix))*np.nanmax(akita_pred_matrix)
            
    return(target_matrix)

In [35]:
def plot_complex_pred(REF_mat, ALT_mat, REF_label, ALT_label, gene_bin_start, gene_bin_end, 
                      gene_len, hic_region, file_path, norm_hic_map, ALT_mat_condensed):
    
    print('plotting ' + ALT_label)
    
    #ref_gene_bin_start=utils.get_bin(utils.half_patch_size - round(gene_len/2))
    #ref_gene_bin_end=utils.get_bin(utils.half_patch_size + round(gene_len/2))
      
    target_index = 0
    vmin=-2; vmax=2
    fig, axes = plt.subplots(2, 3, figsize=(16, 10))

    # Plot 1, raw HiC
    clr = cooler.Cooler(f'{file_path}::resolutions/5000')
    fruitpunch = sns.blend_palette(['white', 'red'], as_cmap=True)

    CHR=hic_region[0]
    seq_start=hic_region[1]
    seq_end=hic_region[2]
    region = (CHR, seq_start, seq_end)
    print('region', region)
    
    im = axes[0,0].matshow(clr.matrix(balance='KR').fetch(region), cmap=fruitpunch, 
                     norm=LogNorm(vmin=1, vmax=3), extent=(seq_start, seq_end, seq_end, seq_start))
    region_str=(f'{CHR}:{seq_start:,}-{seq_end:,}')
    axes[0,0].set_title(f'HiC (5kbp bins) \n chr{region_str}')

    fig.colorbar(im, ax=axes[0,0], fraction=0.046, pad=0.04, label='KR balance')

    # Plot 2, Akita ALT pred
    im = axes[0,1].matshow(ALT_mat, cmap='RdBu_r', vmax=vmax, vmin=vmin)
    axes[0,1].axvline(x=gene_bin_start, color='b', linestyle='--', linewidth=1, label='gene start')
    axes[0,1].axvline(x=gene_bin_end, color='b', linestyle='--', linewidth=1, label='gene end')
    fig.colorbar(im, ax=axes[0,1], fraction=0.04, pad=0.05, ticks=[-2, -1, 0, 1, 2])
    axes[0,1].set_title('Akita ALT Prediction: \n ' + ALT_label)

    # Plot 3, Akita REF pred
    im = axes[0,2].matshow(REF_mat, cmap='RdBu_r', vmax=vmax, vmin=vmin)
    axes[0,2].axvline(x=gene_bin_start, color='b', linestyle='--', linewidth=1, label='gene start')
    axes[0,2].axvline(x=gene_bin_end, color='b', linestyle='--', linewidth=1, label='gene end')
    cbar = fig.colorbar(im, ax=axes[0,2], fraction=0.04, pad=0.05, ticks=[-2, -1, 0, 1, 2])
    axes[0,2].set_title('Akita REF Prediction' + '\n' + REF_label)
    
    
    #Plot 4, HiC normalized to Akita
    im = axes[1,0].matshow(norm_hic_map, cmap= 'RdBu_r', vmax=vmax, vmin=vmin)
    fig.colorbar(im, ax=axes[1,0], fraction=.04, pad = 0.05, ticks=[-2,-1, 0, 1,2]);
    region_str=(f'{CHR}:{seq_start:,}-{seq_end:,}')
    axes[1,0].set_title(f'Normalized HiC (5kbp bins) \n chr{region_str}')

    #Plot 5, Akita prediction with 200 bins total instead of 448 
    im = axes[1,1].matshow(ALT_mat_condensed, cmap= 'RdBu_r', vmax=vmax, vmin=vmin)
    fig.colorbar(im, ax=axes[1,1], fraction=.04, pad = 0.05, ticks=[-2,-1, 0, 1,2]);
    axes[1,1].set_title(f'Akita ALT Prediction (100 bins) \n ' + ALT_label)



    #Plot 6, currently empty    
    im = axes[1,2].matshow(np.zeros((10,10)), cmap= 'RdBu_r', vmax=vmax, vmin=vmin)
    fig.colorbar(im, ax=axes[1,2], fraction=.04, pad = 0.05, ticks=[-2,-1, 0, 1,2]);



    plt.tight_layout()
    plt.savefig('{}{}_Predictions.pdf'.format(fig_dir, ALT_label), dpi=300)   # save the figure to file
    plt.close('all')
    #plt.show()


In [36]:
#PARAMETERS FOR NORMALIZING 

clip=2
crop_bp=0
resolution=5000
#sigma 1, width 5
#stddev is sigma
#kernel_size (which i added) is width, i think 
kernel_stddev=1
#kernel_size=5
pool_width=resolution
no_log=False
crop=False
as_obsexp=True
global_obsexp=False
diagonal_offset=2
#options are KR, VC, VC_SQRT
norm_type='VC_SQRT'
output_file='/pollard/home/shzhang/test1.cool'


def myceil(x, base=resolution):
    return base * np.ceil(x/base)

def myfloor(x, base=resolution):
    return base * np.floor(x/base)

In [37]:
#PARAMETERS FOR TURNING INTO SAME SIZE MATRICES 

# Define the sizes of the original and target matrices
original_size = 448
target_size = 100

# Define the weight for cropping the last row and column by 60%
crop_weight = 0.4 

# Define the kernel bandwidth
bandwidth = 5  # Adjust as needed

In [40]:
for mcool_name in list_mcool:
    
    try:
    
        #make akita prediction
        REF_mat, ALT_mat, R_label, A_label, gene_start, gene_end, gene_len, CHR, seq_start, seq_end = akita_pred(mcool_name)
       
        #need to crop the ends because akita crops the ends 
        crop_ends=((512-448)*2048)/2
        seqs_info=(CHR[3:], myfloor(seq_start+crop_ends), myceil(seq_end-crop_ends))
        hic_region=seqs_info
        file_path=f'{data_dir}mcool/{mcool_name}'
        
        #normalize hic map
        norm_hic_map=normalize_hic(file_path, seqs_info, resolution, clip, crop_bp, kernel_stddev, pool_width, 
                  no_log, crop, as_obsexp, global_obsexp, diagonal_offset, norm_type, output_file)
        #print('target shape', norm_hic_map.shape)
        
        
        #kernel for akita prediction        
        ALT_mat_condensed=condense_matrices(target_size=100, original_size=ALT_mat.shape[0], akita_pred_matrix=ALT_mat)

        
        #kernel for hic map tbd 
               
        
        #plot all plots 
        plot_complex_pred(REF_mat,ALT_mat,R_label,A_label, gene_start, 
                          gene_end, gene_len, hic_region, file_path, norm_hic_map, ALT_mat_condensed)
        
        
        
        
    except:
        print('error!!! in ' + mcool_name)
    
    
    
    
    

SMF-12-23_walk_9_MDM2_containing.fa
starting from left most sequence
/pollard/home/shzhang/akita_variant_scoring/akdemir_collab/data/mcool/SMF-12-23_walk_9_MDM2_containing.fa.mcool::resolutions/5000
ModelSeq(chr='12', start=65000, end=985000, label=None)
(16653,)


/pollard/home/shzhang/miniconda3/envs/akita-working/lib/python3.9/site-packages/cooltools/lib/numutils.py:1376: RuntimeWarning: invalid value encountered in true_divide
  val_cur = ar_cur / armask_cur
/tmp/ipykernel_6359/2476405126.py:32: RuntimeWarning: invalid value encountered in true_divide
  target_matrix=(target_matrix/np.nanmax(target_matrix))*np.nanmax(akita_pred_matrix)
/tmp/ipykernel_6359/2476405126.py:32: RuntimeWarning: All-NaN slice encountered
  target_matrix=(target_matrix/np.nanmax(target_matrix))*np.nanmax(akita_pred_matrix)


plotting SMF-12-23_walk_9_MDM2_containing
region ('12', 65000.0, 985000.0)
SMF-12-23_walk_8_MDM2_containing.fa
ending at rightmost sequence 
/pollard/home/shzhang/akita_variant_scoring/akdemir_collab/data/mcool/SMF-12-23_walk_8_MDM2_containing.fa.mcool::resolutions/5000
ModelSeq(chr='12', start=3340000, end=4265000, label=None)
(16836,)


/pollard/home/shzhang/miniconda3/envs/akita-working/lib/python3.9/site-packages/cooltools/lib/numutils.py:1376: RuntimeWarning: invalid value encountered in true_divide
  val_cur = ar_cur / armask_cur
/tmp/ipykernel_6359/2476405126.py:32: RuntimeWarning: invalid value encountered in true_divide
  target_matrix=(target_matrix/np.nanmax(target_matrix))*np.nanmax(akita_pred_matrix)
/tmp/ipykernel_6359/2476405126.py:32: RuntimeWarning: All-NaN slice encountered
  target_matrix=(target_matrix/np.nanmax(target_matrix))*np.nanmax(akita_pred_matrix)


plotting SMF-12-23_walk_8_MDM2_containing
region ('12', 3340000.0, 4265000.0)
SMF-12-12_walk_3_KRAS_containing.fa
1931914
2980490
4110595
/pollard/home/shzhang/akita_variant_scoring/akdemir_collab/data/mcool/SMF-12-12_walk_3_KRAS_containing.fa.mcool::resolutions/5000
ModelSeq(chr='12', start=1995000, end=2915000, label=None)
(16653,)


...

plotting SMF-12-17_walk_8_MDM2_containing
region ('12', 65000.0, 985000.0)
SMF-12-17_walk_5_GLI1_containing.fa
starting from left most sequence
/pollard/home/shzhang/akita_variant_scoring/akdemir_collab/data/mcool/SMF-12-17_walk_5_GLI1_containing.fa.mcool::resolutions/5000
ModelSeq(chr='12', start=65000, end=985000, label=None)
(16653,)


/pollard/home/shzhang/miniconda3/envs/akita-working/lib/python3.9/site-packages/cooler/api.py:744: RuntimeWarning: divide by zero encountered in true_divide
  bias1 = 1 / bias1
/pollard/home/shzhang/miniconda3/envs/akita-working/lib/python3.9/site-packages/cooler/api.py:745: RuntimeWarning: divide by zero encountered in true_divide
  bias2 = 1 / bias2
/pollard/home/shzhang/miniconda3/envs/akita-working/lib/python3.9/site-packages/cooler/api.py:746: RuntimeWarning: invalid value encountered in multiply
  arr = arr * np.outer(bias1, bias2)
/pollard/home/shzhang/miniconda3/envs/akita-working/lib/python3.9/site-packages/cooltools/lib/numutils.py:1376: RuntimeWarning: invalid value encountered in true_divide
  val_cur = ar_cur / armask_cur
/tmp/ipykernel_6359/2476405126.py:32: RuntimeWarning: invalid value encountered in true_divide
  target_matrix=(target_matrix/np.nanmax(target_matrix))*np.nanmax(akita_pred_matrix)
/tmp/ipykernel_6359/2476405126.py:32: RuntimeWarning: All-NaN slice encoun

plotting SMF-12-17_walk_5_GLI1_containing
region ('12', 65000.0, 985000.0)
SMF-12-17_walk_3_GLI1_containing.fa
917253
1965829
2201610
/pollard/home/shzhang/akita_variant_scoring/akdemir_collab/data/mcool/SMF-12-17_walk_3_GLI1_containing.fa.mcool::resolutions/5000
ModelSeq(chr='12', start=980000, end=1905000, label=None)
(16836,)


/pollard/home/shzhang/miniconda3/envs/akita-working/lib/python3.9/site-packages/cooler/api.py:744: RuntimeWarning: divide by zero encountered in true_divide
  bias1 = 1 / bias1
/pollard/home/shzhang/miniconda3/envs/akita-working/lib/python3.9/site-packages/cooler/api.py:745: RuntimeWarning: divide by zero encountered in true_divide
  bias2 = 1 / bias2
/pollard/home/shzhang/miniconda3/envs/akita-working/lib/python3.9/site-packages/cooler/api.py:746: RuntimeWarning: invalid value encountered in multiply
  arr = arr * np.outer(bias1, bias2)
/pollard/home/shzhang/miniconda3/envs/akita-working/lib/python3.9/site-packages/cooltools/lib/numutils.py:1376: RuntimeWarning: invalid value encountered in true_divide
  val_cur = ar_cur / armask_cur
/tmp/ipykernel_6359/2476405126.py:32: RuntimeWarning: invalid value encountered in true_divide
  target_matrix=(target_matrix/np.nanmax(target_matrix))*np.nanmax(akita_pred_matrix)
/tmp/ipykernel_6359/2476405126.py:32: RuntimeWarning: All-NaN slice encoun

plotting SMF-12-17_walk_3_GLI1_containing
region ('12', 980000.0, 1905000.0)


In [ ]:
2980490-1931914

In [ ]:

    original_size = 448
    target_size = 100

    # Define the weight for cropping the last row and column by 60%
    crop_weight = 0.4 


    # Define the kernel bandwidth
    bandwidth = 10  # Adjust as needed

In [ ]:
import pdg

In [ ]:
#let's plot
#original, unwhatevered mcool
#mcool with normalization
#akita prediction, with lines for genes DONE 

#underneath, plot windowed prediction with same number of bins 
#mcool
#akita prediction 